In [3]:
#載入LineBot所需要的模組
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *

In [4]:
app = Flask(__name__)

# 必須放上自己的Channel Access Token
line_bot_api = LineBotApi('plMcM04uPLPHEsBYPqZZtFHj0pM9I19dOVByC5fp7FZf4gFTW83BZVWYBcTW/g/ENqoWfTrIUx6tLCyAVLRuGh4At1PXF2GZPqhTvswClAZCphOFNRgDyAvobPNymbm4D04ZsOAxaq8d9wNT6ojOxwdB04t89/1O/w1cDnyilFU=')

# 必須放上自己的Channel Secret
handler = WebhookHandler('fa28c66980cf43dd8b1e41630b2f8fd1')

In [6]:
line_bot_api.push_message('U82e65d3ed7880828c259a6af649abc96', TextSendMessage(text='你可以開始了'))

In [7]:
# 監聽所有來自 /callback 的 Post Request
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [8]:
#訊息傳遞區塊
##### 基本上程式編輯都在這個function #####
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    message = TextSendMessage(text=event.message.text)
    line_bot_api.reply_message(event.reply_token,message)

In [12]:
#主程式
import os 
if (__name__ == "__main__"):
    port = int(os.environ.get('PORT', 400))
    app.run(host='0.0.0.0', port=port)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:400/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Oct/2022 14:09:02] "POST /callback HTTP/1.1" 200 -
